№3
 Задание №1
  

In [144]:
import numpy as np
from scipy import stats
from statsmodels.stats.diagnostic import lilliefors

In [145]:
sample1 = np.random.normal(loc=0, scale=1, size=100)
sample2 = np.random.normal(loc=0, scale=1, size=100)

sample3 = np.random.normal(loc=0, scale=1, size=100)
sample4 = np.random.exponential(size=100)

sample5 = stats.binom.rvs(10, .5, size=100)
sample6 = stats.uniform.rvs(0, 1, size=100)

Выдвинем и проверим на заданном уровне значимости 𝛼 для каждой 
пары выборок гипотезы

a) О равенстве двух дисперсий 

In [146]:
alpha = 0.05 #Уровень значимости

variance1 = np.var(sample1, ddof=1)
variance2 = np.var(sample2, ddof=1)

Выполним проверку выборок на нормальность с помощью критерия Шаппиро


In [147]:
def normal_check(sample):
    _, p_value = stats.shapiro(sample)
    if p_value < alpha:
        return False
    else:
        return True

Выполним проверку гипотезы на равенство дисперсий

In [148]:
samples = [[sample1, sample2], [sample3, sample4], [sample5, sample6]]
def variance_check(samples):
    for sample in samples:
        if normal_check(sample[0]) and normal_check(sample[1]):
            _, p_value = stats.bartlett(sample[0], sample[1])
            if p_value < 0.05:
                print("Гипотеза о равенстве дисперсий отклоняется")
            else:
                print("Гипотеза о равенстве дисперсий не отклоняется")
        else:
            _, p_value = stats.levene(sample[0], sample[1])
            if p_value < 0.05:
                print("Гипотеза о равенстве дисперсий отклоняется")
            else:
                print("Гипотеза о равенстве дисперсий не отклоняется")
    
variance_check(samples)

Гипотеза о равенстве дисперсий не отклоняется
Гипотеза о равенстве дисперсий отклоняется
Гипотеза о равенстве дисперсий отклоняется


b) О равенстве двух выборочных средних
Проверка выборок на нормальность с помощью критерия Шаппиро-Уилка

In [149]:
def mean_check(samples):
    for sample in samples:
        if normal_check(sample[0]) and normal_check(sample[1]):
            _, p_value = stats.ttest_ind(sample[0], sample[1])
            if p_value < 0.05:
                print("Гипотеза о равенстве средних значений отклоняется")
            else:
                print("Гипотеза о равенстве средних значений не отклоняется")
        else:
            _, p_value = stats.mannwhitneyu(sample[0], sample[1])
            if p_value < 0.05:
                print("Гипотеза о равенстве средних значений отклоняется")
            else:
                print("Гипотеза о равенстве средних значений не отклоняется")
    
mean_check(samples)

Гипотеза о равенстве средних значений не отклоняется
Гипотеза о равенстве средних значений отклоняется
Гипотеза о равенстве средних значений отклоняется


Здание №2

In [150]:
N_values = [100, 300, 500, 1000] 
num_samples = 1000 
normal_samples = stats.norm.rvs(size=10)
expon_samples = stats.expon.rvs(size=10)

Функции для вычисления мощности критерия

In [151]:
def calculate_ks_power(n, distribution=stats.norm, alt_distribution=stats.norm, alpha=0.05, iterations=1000, **dist_params):
    power = 0
    for _ in range(iterations):
        sample = alt_distribution.rvs(size=n, **dist_params)
        _, p_value = stats.kstest(sample, distribution.cdf)
        if p_value < alpha:
            power += 1
    return power / iterations

def calculate_anderson_power(n, distribution=stats.norm, alt_distribution=stats.norm, alpha=0.05, iterations=1000, **dist_params):
    power = 0
    for _ in range(iterations):
        sample = alt_distribution.rvs(size=n, **dist_params)
        
        result = stats.anderson(sample, dist=distribution.name)
        
        test_statistic = result.statistic

        if test_statistic > result.critical_values[2]:  # Выбрано критическое значение для уровня значимости 0.05
            power += 1
    
    power = power / iterations
    
    return power

def calculate_sw_power(n, alt_distribution=stats.norm, alpha=0.05, iterations=1000, **dist_params):
    power = 0
    for _ in range(iterations):
        sample = alt_distribution.rvs(size=n, **dist_params)
        _, p_value = stats.shapiro(sample)
        if p_value < alpha:
            power += 1
    return power / iterations

def calculate_lil_power(n, alt_distribution=stats.norm, alpha=0.05, iterations=1000, **dist_params):
    power = 0
    for _ in range(iterations):

        sample = alt_distribution.rvs(size=n, **dist_params)
        _, p_value = lilliefors(sample)
        if p_value < alpha:
            power += 1
    return power / iterations

Для нормального распределения

In [152]:
for N in N_values:
    for _ in range(1000):
        normal_samples = stats.norm.rvs(size = 100)
    print(f"Для выборок размером {N}")
    norm_ad_stat, norm_ad_critical_values, norm_ad_significance_levels = stats.anderson(normal_samples)
    norm_lf_stat, norm_lf_p_values = lilliefors(normal_samples)
    norm_ks_stat, norm_ks_p_values = stats.kstest(normal_samples, 'norm')
    norm_sw_stat, norm_sw_p_values = stats.shapiro(normal_samples)
    
    print("Для нормального распределения")
    
    print("Метод Anderson-Darling:")
    print("Статистика:", norm_ad_stat)
    print("Критические значения:", norm_ad_critical_values)
    print("Уровни значимости:", norm_ad_significance_levels)
    norm_ad_power = calculate_anderson_power(N, alt_distribution=stats.laplace, alpha=alpha, iterations=1000)
    print("Мощность критерия Anderson-Darling: ", norm_ad_power)
    print("-" * 20)
    
    print("Метод Lilliefors:")
    print("Статистика:", norm_lf_stat)
    print("p-значения:", norm_lf_p_values)
    norm_lf_power = calculate_lil_power(N, alt_distribution=stats.laplace, alpha=alpha, iterations=1000)
    print("Мощность критерия Lilliefors: ", norm_lf_power)
    print("-" * 20)
    
    print("Метод Kolmogorov-Smirnov:")
    print("Статистика:", norm_ks_stat)
    print("p-значения:", norm_ks_p_values)
    norm_ks_power = calculate_ks_power(N, alt_distribution=stats.laplace, alpha=alpha, iterations=1000)
    print("Мощность критерия Kolmogorov-Smirnov: ", norm_ks_power)
    print("-" * 20)
    
    print("Метод Shapiro-Wilk:")
    print("Статистика:", norm_sw_stat)
    print("p-значения:", norm_sw_p_values)
    norm_sw_power = calculate_sw_power(N, alt_distribution=stats.laplace, alpha=alpha, iterations=1000)
    print("Мощность критерия Shapiro-Wilk: ", norm_sw_power)
    print("-" * 20)


Для выборок размером 100
Для нормального распределения
Метод Anderson-Darling:
Статистика: 0.20900258066924948
Критические значения: [0.555 0.632 0.759 0.885 1.053]
Уровни значимости: [15.  10.   5.   2.5  1. ]
Мощность критерия Anderson-Darling:  0.833
----------------------------
Метод Lilliefors:
Статистика: 0.04891507052328936
p-значения: 0.8053594166239361
Мощность критерия Lilliefors:  0.72
----------------------------
Метод Kolmogorov-Smirnov:
Статистика: 0.07089282018289678
p-значения: 0.6696155454186286
Мощность критерия Kolmogorov-Smirnov:  0.081
----------------------------
Метод Shapiro-Wilk:
Статистика: 0.9945318102836609
p-значения: 0.9611434936523438
Мощность критерия Shapiro-Wilk:  0.809
----------------------------
Для выборок размером 300
Для нормального распределения
Метод Anderson-Darling:
Статистика: 0.3964340695154789
Критические значения: [0.555 0.632 0.759 0.885 1.053]
Уровни значимости: [15.  10.   5.   2.5  1. ]
Мощность критерия Anderson-Darling:  0.999
-----

Для экспоненциального распределения

In [153]:
for N in N_values:
    for _ in range(1000):
        expon_samples = stats.norm.rvs(size = 100)
    print(f"Для выборок размером {N}")
    ex_ad_stat, ex_ad_critical_values, ex_ad_significance_levels = stats.anderson(expon_samples)
    ex_lf_stat, ex_lf_p_values = lilliefors(expon_samples)
    ex_ks_stat, ex_ks_p_values = stats.kstest(expon_samples, "expon")
    ex_sw_stat, ex_sw_p_values = stats.shapiro(expon_samples)
    
    print("Для экспоненциального распределения")
    print("Метод Anderson-Darling:")
    print("Статистика:", ex_ad_stat)
    print("Критические значения:", ex_ad_critical_values)
    print("Уровни значимости:", ex_ad_significance_levels)
    ex_ad_power = calculate_anderson_power(N, alt_distribution=stats.laplace, alpha=alpha, iterations=1000)
    print("Мощность критерия Anderson-Darling: ", ex_ad_power)
    print("-" * 20)
    
    print("Метод Lilliefors:")
    print("Статистика:", ex_lf_stat)
    print("p-значения:", ex_lf_p_values)
    ex_sw_power = calculate_lil_power(N, alt_distribution=stats.laplace, alpha=alpha, iterations=1000)
    print("Мощность критерия Lilliefors: ", ex_sw_power)
    print("-" * 20)
    
    print("Метод Kolmogorov-Smirnov:")
    print("Статистика:", ex_ks_stat)
    print("p-значения:", ex_ks_p_values)
    ex_ks_power = calculate_ks_power(N, distribution=stats.expon, alt_distribution=stats.laplace, alpha=alpha, iterations=1000)
    print("Мощность критерия Kolmogorov-Smirnov: ", ex_ks_power)
    print("-" * 20)
    
    print("Метод Shapiro-Wilk:")
    print("Статистика:", ex_sw_stat)
    print("p-значения:", ex_sw_p_values)
    ex_sw_power = calculate_sw_power(N, alt_distribution=stats.laplace, alpha=alpha, iterations=1000)
    print("Мощность критерия Shapiro-Wilk: ", ex_sw_power)
    print("-" * 20)
    

Для выборок размером 100
Для экспоненциального распределения
Метод Anderson-Darling:
Статистика: 0.30126155251100784
Критические значения: [0.555 0.632 0.759 0.885 1.053]
Уровни значимости: [15.  10.   5.   2.5  1. ]
Мощность критерия Anderson-Darling:  0.831
----------------------------
Метод Lilliefors:
Статистика: 0.05780710540841333
p-значения: 0.5655905508971847
Мощность критерия Lilliefors:  0.714
----------------------------
Метод Kolmogorov-Smirnov:
Статистика: 0.5455130285179957
p-значения: 1.969164093217698e-28
Мощность критерия Kolmogorov-Smirnov:  1.0
----------------------------
Метод Shapiro-Wilk:
Статистика: 0.9912046194076538
p-значения: 0.7610797882080078
Мощность критерия Shapiro-Wilk:  0.797
----------------------------
Для выборок размером 300
Для экспоненциального распределения
Метод Anderson-Darling:
Статистика: 0.3974067751138932
Критические значения: [0.555 0.632 0.759 0.885 1.053]
Уровни значимости: [15.  10.   5.   2.5  1. ]
Мощность критерия Anderson-Darling:

Мощность критерия зависит от вида распределения, размера выборки и метода

Вывод: изучены методы проверки статистических гипотез